In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import mlflow
import mlflow.sklearn


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Dropout, Flatten


In [3]:

# Set up MLflow
mlflow.set_experiment("Fraud Detection")


<Experiment: artifact_location='file:///c:/Users/Firew%20Ayele/Desktop/kifiya/Adey-Innovations-Inc/notebooks/mlruns/969604978361663413', creation_time=1729609485785, experiment_id='969604978361663413', last_update_time=1729609485785, lifecycle_stage='active', name='Fraud Detection', tags={}>

Load the datasets

In [4]:
# Load the datasets
fraud_data = pd.read_csv('../data/Fraud_Data.csv')
credit_card_data = pd.read_csv('../data/creditcard.csv')

In [5]:
fraud_data

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0
...,...,...,...,...,...,...,...,...,...,...,...
151107,345170,2015-01-27 03:03:34,2015-03-29 00:30:47,43,XPSKTWGPWINLR,SEO,Chrome,M,28,3.451155e+09,1
151108,274471,2015-05-15 17:43:29,2015-05-26 12:24:39,35,LYSFABUCPCGBA,SEO,Safari,M,32,2.439047e+09,0
151109,368416,2015-03-03 23:07:31,2015-05-20 07:07:47,40,MEQHCSJUBRBFE,SEO,IE,F,26,2.748471e+09,0
151110,207709,2015-07-09 20:06:07,2015-09-07 09:34:46,46,CMCXFGRHYSTVJ,SEO,Chrome,M,37,3.601175e+09,0


In [6]:
credit_card_data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [7]:
# Inspect columns of fraud_data
print("Columns in fraud_data:", fraud_data.columns)

Columns in fraud_data: Index(['user_id', 'signup_time', 'purchase_time', 'purchase_value',
       'device_id', 'source', 'browser', 'sex', 'age', 'ip_address', 'class'],
      dtype='object')


Convert datetime strings to datetime objects

In [8]:
# Convert datetime strings to datetime objects
fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])


Extract useful datetime components

In [ ]:

# Extract useful datetime components
fraud_data['signup_hour'] = fraud_data['signup_time'].dt.hour
fraud_data['signup_day'] = fraud_data['signup_time'].dt.dayofweek
fraud_data['purchase_hour'] = fraud_data['purchase_time'].dt.hour
fraud_data['purchase_day'] = fraud_data['purchase_time'].dt.dayofweek

# Drop the original datetime columns
fraud_data = fraud_data.drop(columns=['signup_time', 'purchase_time'])

Preprocess Fraud_Data.csv

In [9]:
# Preprocess Fraud_Data.csv
X_fraud = fraud_data.drop(columns=['class'])
y_fraud = fraud_data['class']


Preprocess creditcard.csv

In [10]:
# Preprocess creditcard.csv
X_cc = credit_card_data.drop(columns=['Class'])
y_cc = credit_card_data['Class']

Train-test split

In [11]:
# Train-test split
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud)
X_train_cc, X_test_cc, y_train_cc, y_test_cc = train_test_split(X_cc, y_cc, test_size=0.2, random_state=42, stratify=y_cc)


In [12]:
# Convert datetime columns if any
if 'TransactionDate' in X_train_fraud.columns:  # Replace with your datetime column
    X_train_fraud['TransactionDate'] = pd.to_datetime(X_train_fraud['TransactionDate'])
    X_train_fraud['TransactionYear'] = X_train_fraud['TransactionDate'].dt.year
    X_train_fraud['TransactionMonth'] = X_train_fraud['TransactionDate'].dt.month
    X_train_fraud['TransactionDay'] = X_train_fraud['TransactionDate'].dt.day
    X_train_fraud['TransactionHour'] = X_train_fraud['TransactionDate'].dt.hour
    # Drop the original datetime column
    X_train_fraud = X_train_fraud.drop('TransactionDate', axis=1)

# Repeat for X_test_fraud if needed

# Select only numeric columns for scaling
numeric_cols_fraud = X_train_fraud.select_dtypes(include=['float64', 'int64']).columns
numeric_cols_cc = X_train_cc.select_dtypes(include=['float64', 'int64']).columns


Feature scaling (standardization)

In [ ]:

# Feature scaling (standardization)
scaler = StandardScaler()

X_train_fraud_scaled = scaler.fit_transform(X_train_fraud[numeric_cols_fraud])
X_test_fraud_scaled = scaler.transform(X_test_fraud[numeric_cols_fraud])

X_train_cc_scaled = scaler.fit_transform(X_train_cc[numeric_cols_cc])
X_test_cc_scaled = scaler.transform(X_test_cc[numeric_cols_cc])


Define Evaluation Function

In [13]:
# Define Evaluation Function
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=1)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    return accuracy, precision, recall, f1, roc_auc


In [14]:
def log_metrics(model_name, accuracy, precision, recall, f1, roc_auc):
    mlflow.log_param("Model", model_name)
    mlflow.log_metric("Accuracy", accuracy)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1 Score", f1)
    mlflow.log_metric("ROC-AUC", roc_auc)


Train and Evaluate Models

In [15]:
# Train and Evaluate Models
def train_and_evaluate(model, X_train, X_test, y_train, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy, precision, recall, f1, roc_auc = evaluate_model(y_test, y_pred)
    
    with mlflow.start_run():
        mlflow.sklearn.log_model(model, model_name)
        log_metrics(model_name, accuracy, precision, recall, f1, roc_auc)
        
    print(f"{model_name} - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}, ROC-AUC: {roc_auc}")


In [16]:
# Initialize models
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(),
    'MLP': MLPClassifier(max_iter=500)
}

In [17]:
# Check for non-numeric columns
print(X_train_fraud.dtypes)


user_id             int64
purchase_value      int64
device_id          object
source             object
browser            object
sex                object
age                 int64
ip_address        float64
signup_hour         int32
signup_day          int32
purchase_hour       int32
purchase_day        int32
dtype: object


In [18]:
from sklearn.preprocessing import LabelEncoder

# Label encode 'sex' column
label_encoder = LabelEncoder()
X_train_fraud['sex'] = label_encoder.fit_transform(X_train_fraud['sex'])
X_test_fraud['sex'] = label_encoder.transform(X_test_fraud['sex'])

In [19]:
# Frequency encoding for 'device_id', 'source', and 'browser'
for col in ['device_id', 'source', 'browser']:
    freq_encoding = X_train_fraud[col].value_counts(normalize=True)
    X_train_fraud[col] = X_train_fraud[col].map(freq_encoding)
    X_test_fraud[col] = X_test_fraud[col].map(freq_encoding)


In [20]:
from category_encoders import TargetEncoder

# Target encode 'device_id', 'source', and 'browser'
target_encoder = TargetEncoder(cols=['device_id', 'source', 'browser'])
X_train_fraud = target_encoder.fit_transform(X_train_fraud, y_train_fraud)
X_test_fraud = target_encoder.transform(X_test_fraud)


In [21]:
# Encode only the top N frequent categories and group others
N = 10  # Choose the number of top categories to keep

for col in ['device_id', 'source', 'browser']:
    top_categories = X_train_fraud[col].value_counts().nlargest(N).index
    X_train_fraud[col] = X_train_fraud[col].where(X_train_fraud[col].isin(top_categories), other='Other')
    X_test_fraud[col] = X_test_fraud[col].where(X_test_fraud[col].isin(top_categories), other='Other')

# Then, apply one-hot encoding after reducing categories
X_train_fraud = pd.get_dummies(X_train_fraud, columns=['device_id', 'source', 'browser'], drop_first=True)
X_test_fraud = pd.get_dummies(X_test_fraud, columns=['device_id', 'source', 'browser'], drop_first=True)



Train and evaluate on Fraud_Data.csv and creditcard.csv

In [22]:
# Train and evaluate on Fraud_Data.csv
for model_name, model in models.items():
    train_and_evaluate(model, X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud, model_name)

# Train and evaluate on creditcard.csv
for model_name, model in models.items():
    train_and_evaluate(model, X_train_cc, X_test_cc, y_train_cc, y_test_cc, model_name)


2024/10/24 20:52:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logistic Regression - Accuracy: 0.9063627039010026, Precision: 1.0, Recall: 0.0, F1: 0.0, ROC-AUC: 0.5


2024/10/24 20:52:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Decision Tree - Accuracy: 0.6250206796148629, Precision: 0.15125912558444754, Recall: 0.6515901060070671, F1: 0.24552293455828506, ROC-AUC: 0.6369329349441754


2024/10/24 20:53:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Random Forest - Accuracy: 0.9996137776061234, Precision: 0.9318181818181818, Recall: 0.8367346938775511, F1: 0.8817204301075269, ROC-AUC: 0.9183145894823884


2024/10/24 21:18:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Gradient Boosting - Accuracy: 0.9983146659176293, Precision: 0.5294117647058824, Recall: 0.1836734693877551, F1: 0.2727272727272727, ROC-AUC: 0.5916960481435117


2024/10/24 21:20:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MLP - Accuracy: 0.99875355500158, Precision: 0.6666666666666666, Recall: 0.5510204081632653, F1: 0.6033519553072626, ROC-AUC: 0.7752727955278903


Convolutional Neural Network (CNN)

In [25]:
# Convolutional Neural Network (CNN)
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Flatten())  # Flatten the output from Conv1D before feeding into Dense layers
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # For binary classification
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [27]:
# Ensure y_train_fraud and y_test_fraud are NumPy arrays with the correct dtype
y_train_fraud = np.array(y_train_fraud, dtype='float32')
y_test_fraud = np.array(y_test_fraud, dtype='float32')

# Convert the DataFrame to a NumPy array and reshape it for CNN input
X_train_fraud_reshaped = X_train_fraud.values.reshape(-1, X_train_fraud.shape[1], 1).astype('float32')
X_test_fraud_reshaped = X_test_fraud.values.reshape(-1, X_test_fraud.shape[1], 1).astype('float32')

# Build and train the CNN model
cnn_model = build_cnn_model((X_train_fraud.shape[1], 1))
cnn_model.fit(X_train_fraud_reshaped, y_train_fraud, epochs=10, batch_size=64)

# Make predictions on the test set
y_pred_cnn = cnn_model.predict(X_test_fraud_reshaped)
y_pred_cnn = (y_pred_cnn > 0.5).astype(int)

# Evaluate the model
accuracy, precision, recall, f1, roc_auc = evaluate_model(y_test_fraud, y_pred_cnn)

# Log the model and metrics using MLflow
with mlflow.start_run():
    mlflow.keras.log_model(cnn_model, "CNN")
    log_metrics("CNN", accuracy, precision, recall, f1, roc_auc)


# Print the evaluation results
print(f"CNN - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}, ROC-AUC: {roc_auc}")


Epoch 1/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8300 - loss: 3249569.2500
Epoch 2/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8252 - loss: 688053.0000
Epoch 3/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8288 - loss: 232842.1562
Epoch 4/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8300 - loss: 101171.6094
Epoch 5/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8318 - loss: 62328.0898
Epoch 6/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8307 - loss: 13228.5732
Epoch 7/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8319 - loss: 10474.9355
Epoch 8/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8282 - loss: 2936.4158
Epoch 9/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8346 - loss: 564.8339
Epoch 10/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9061 - loss: 0.3718
945/945 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


2024/10/24 21:28:59 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/24 21:29:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


CNN - Accuracy: 0.9063627039010026, Precision: 1.0, Recall: 0.0, F1: 0.0, ROC-AUC: 0.5


Build and train the LSTM model

In [29]:
# Convert DataFrames to NumPy arrays and reshape
X_train_fraud_reshaped = X_train_fraud.values.reshape(-1, X_train_fraud.shape[1], 1).astype('float32')
X_test_fraud_reshaped = X_test_fraud.values.reshape(-1, X_test_fraud.shape[1], 1).astype('float32')

# Ensure the target variables are NumPy arrays with the correct dtype
y_train_fraud = np.array(y_train_fraud, dtype='float32')
y_test_fraud = np.array(y_test_fraud, dtype='float32')

# Build and train the LSTM model
lstm_model = build_lstm_model((X_train_fraud.shape[1], 1))
lstm_model.fit(X_train_fraud_reshaped, y_train_fraud, epochs=10, batch_size=64)

# Make predictions on the test set
y_pred_lstm = lstm_model.predict(X_test_fraud_reshaped)
y_pred_lstm = (y_pred_lstm > 0.5).astype(int)

# Evaluate the model
accuracy, precision, recall, f1, roc_auc = evaluate_model(y_test_fraud, y_pred_lstm)

# Log the model and metrics using MLflow
with mlflow.start_run():
    mlflow.keras.log_model(lstm_model, "LSTM")
    log_metrics("LSTM", accuracy, precision, recall, f1, roc_auc)

# Print the evaluation results
print(f"LSTM - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}, ROC-AUC: {roc_auc}")


Epoch 1/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 58s 29ms/step - accuracy: 0.9101 - loss: 0.3043
Epoch 2/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 62s 33ms/step - accuracy: 0.9479 - loss: 0.1922
Epoch 3/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 63s 33ms/step - accuracy: 0.9488 - loss: 0.1870
Epoch 4/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 60s 32ms/step - accuracy: 0.9495 - loss: 0.1856
Epoch 5/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 59s 31ms/step - accuracy: 0.9504 - loss: 0.1838
Epoch 6/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 57s 30ms/step - accuracy: 0.9504 - loss: 0.1835
Epoch 7/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 58s 31ms/step - accuracy: 0.9510 - loss: 0.1813
Epoch 8/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 56s 30ms/step - accuracy: 0.9508 - loss: 0.1824
Epoch 9/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 60s 31ms/step - accuracy: 0.9531 - loss: 0.1759
Epoch 10/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 61s 32ms/step - accuracy: 0.9531 - loss: 0.1765
945/945 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step


2024/10/24 21:46:18 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/24 21:46:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


LSTM - Accuracy: 0.839096052675115, Precision: 0.3059744225997328, Recall: 0.5664310954063604, F1: 0.3973230883628702, ROC-AUC: 0.7168482275849017
